In [1]:
# System
import os
from dotenv import load_dotenv
import ast
import random

# LLM Models
from langchain_openai import ChatOpenAI
# from openai import OpenAI

# Template
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [2]:
load_dotenv()

True

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=8192,
                        api_key=os.getenv("OPENAI_API_KEY"),
                        temperature=0.7)

In [4]:
questions = [
    {"Question": "Question 1", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 2", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 3", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 4", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 5", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 6", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 7", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 8", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 9", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 10", "Choices": ["A", "B", "C", "D"], "Answer": "A"}
]

In [5]:
def questions_import(exam_questions='Bible'):

    if exam_questions == 'Bible':
        from data.bible_questions import questions
        result = random.sample(questions, 10)
    elif exam_questions == 'American Mathematics Competitions 8 (AMC 8)':
        from data.amc8_questions import questions
        result = random.sample(questions, 10)
    elif exam_questions == 'American Mathematics Competitions 10 (AMC 10)':
        from data.amc10_questions import questions
        result = random.sample(questions, 10)
    elif exam_questions == 'Vocabulary (6th Grade)':
        from data.vocabulary_questions import questions as database_questions
        vocab = [word for word in database_questions if database_questions[word] == 6]
        vocabulary_questions = random.sample(vocab, 10)
        question_content = vocabulary_generator(vocabulary_questions)
        result = extract_questions(question_content, randomize=True)
    elif exam_questions == 'Vocabulary (8th Grade)':
        from data.vocabulary_questions import questions as database_questions
        vocab = [word for word in database_questions if database_questions[word] == 8]
        vocabulary_questions = random.sample(vocab, 10)
        question_content = vocabulary_generator(vocabulary_questions)
        result = extract_questions(question_content, randomize=True)
    elif exam_questions == 'Vocabulary (10th Grade)':
        from data.vocabulary_questions import questions as database_questions
        vocab = [word for word in database_questions if database_questions[word] == 10]
        vocabulary_questions = random.sample(vocab, 10)
        question_content = vocabulary_generator(vocabulary_questions)
        result = extract_questions(question_content, randomize=True)
    elif exam_questions == 'Vocabulary (12th Grade)':
        from data.vocabulary_questions import questions as database_questions
        vocab = [word for word in database_questions if database_questions[word] == 12]
        vocabulary_questions = random.sample(vocab, 10)
        question_content = vocabulary_generator(vocabulary_questions)
        result = extract_questions(question_content, randomize=True)
    elif exam_questions == 'AP US History':
        from data.apushistory_questions import questions
        result = random.sample(questions, 10)

    return result

In [6]:
def extract_questions(content, randomize=False):
    result = list()
    questions = content.split('---')
    counter = 1

    for ques in questions:
        parts = ques.split('\n')
        q = c = a = e = d = s = ""
        for part in parts:
            if part.startswith('Question:'):
                # q = f'Question {counter}: <br />' + part.split('Question: ')[1].rstrip(' ')
                q = part.split('Question: ')[1].rstrip(' ')
            elif part.startswith('Choices:'):
                c = ast.literal_eval(part.split('Choices: ')[1].rstrip(' '))
                if randomize:
                    random.shuffle(c)
            elif part.startswith('Answer:'):
                a = part.split('Answer: ')[1].rstrip(' ').rstrip('"').lstrip('"')
            elif part.startswith('Explanation:'):
                e = part.split('Explanation: ')[1].rstrip(' ').rstrip('"').lstrip('"')
            elif part.startswith('Definition:'):
                d = part.split('Definition: ')[1].rstrip(' ').rstrip('"').lstrip('"')
            elif part.startswith('Sentence:'):
                s = part.split('Sentence: ')[1].rstrip(' ').rstrip('"').lstrip('"')
        if q != "":
            result.append({"Question": q, "Choices": c, "Answer": a, "Explanation": e, "Definition": d, "Sentence": s})
            counter += 1

    return result

In [7]:
def question_generator(subject, grade):

    prompt = ChatPromptTemplate.from_template(
        """
        Generate a multiple-choice exam with 10 questions 
        on {subject} for a {grade} grade level student.
        Structure each question in the following format:
        Question: What is the capital of France?
        Choices: ["London", "Paris", "Rome", "Berlin"]
        Answer: "Paris"
        Separate each question by '---'.
        Do not surround 'Question' with double asterisk '**'.
        """
    )

    chat_chain = prompt | chat_model | StrOutputParser()
    response = chat_chain.invoke({"subject": subject, "grade": grade})

    return response

In [8]:
def vocabulary_generator(words):

    prompt = ChatPromptTemplate.from_template(
        """
        Generate multiple-choice question, answer, and explanation on the definition
        for each word in {words}.
        Structure each question in the following format:
        Question: What is the meaning of benevolent?
        Choices: ["Harmful", "Kind and generous", "Dangerous", "Uninterested"]
        Answer: "Kind and generous"
        Definition: "an adjective that describes someone who is well-meaning and kindly, 
        often characterized by a desire to help others or promote their welfare"
        Sentence: "The benevolent woman spent her weekends volunteering at the local shelter, 
        providing food and support to those in need."
        Separate each question by '---'.
        Do not surround 'Question' with double asterisk '**'.
        """
    )

    chat_chain = prompt | chat_model | StrOutputParser()
    response = chat_chain.invoke({"words": words})

    return response

In [9]:
def exam_generator1(subject, grade):

    global questions

    questions_text = question_generator(subject, grade)
    questions = extract_questions(questions_text)
    result = []
    counter = 1

    for question in questions:
        result.append(gr.Markdown(f'Question {counter}: <br />' + question["Question"],
                                  latex_delimiters=[{"left": "$$/", "right": "/$$", "display": False}],
                                  ))
        result.append(gr.Radio(choices=question["Choices"],
                               value=None,
                               label=question["Question"],
                               show_label=False,
                               container=True,
                               ))
        counter += 1

    return result

In [10]:
def exam_generator2(exam_questions='Bible'):

    global questions

    questions = questions_import(exam_questions=exam_questions)
    inputs = []

    counter = 1
    for question in questions:
        if "Source" not in question:
            inputs.append(gr.Markdown(f'Question {counter}: <br />' + question["Question"],
                                      latex_delimiters=[{"left": "$$/", "right": "/$$", "display": False}],
                                      ))
        else:
            inputs.append(gr.Markdown(f'Question {counter}: <br />' + question["Question"] + question["Source"],
                                      latex_delimiters=[{"left": "$$/", "right": "/$$", "display": False}],
                                      ))
        inputs.append(gr.Radio(choices=question["Choices"],
                               value=None,
                               label=(f'Question {counter}: ' + question["Question"]),
                               show_label=False,
                               container=True,
                               ))
        counter += 1

    return inputs

In [11]:
def grade_exam(*answers):
    wrong_answers = ''
    score = 0
    for i, question in enumerate(questions):
        if answers[i] == question["Answer"]:
            score += 1
        else:
            wrong_answers += f'**Question {i + 1}:** <br />{question["Question"]}<br />\tYour answer is: {answers[i]}. <br />\n\tThe correct answer is: {question["Answer"]}.'
            if 'Explanation' in question:
                if question["Explanation"] != "":
                    wrong_answers += f'<br />\tExplanation: {question["Explanation"]}'
            if 'Definition' in question:
                if question["Definition"] != "":
                    wrong_answers += f'<br />\tDefinition: {question["Definition"]}'
            if 'Sentence' in question:
                if question["Sentence"] != "":
                    wrong_answers += f'<br />\tSentence: {question["Sentence"]}'
            wrong_answers += '<br /><br />'

    total_questions = len(questions)

    if wrong_answers == "":
        result = f"**You scored {score} out of {total_questions}.**\n**Perfect Score. Congratulations.**"
    else:
        result = f"**You scored {score} out of {total_questions}.**\n\n**Questions that you got wrong:**\n\n" + wrong_answers

    return result

In [12]:
css_template = """                   
               h1 {
                   font-size: 36px;
                   font-family: Verdana, Arial, Sans-Serif;
                   font-style: normal;
                   font-weight: bold;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: center;
                   text-transform: normal;
                   letter-spacing: 2px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }
                   
               h2 {
                   font-size: 24px;
                   font-family: Verdana, Arial, Sans-Serif;
                   font-style: normal;
                   font-weight: bold;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: center;
                   text-transform: normal;
                   letter-spacing: 2px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }

               p {
                   font-size: 14px;
                   font-family: Times New Roman, Times;
                   font-style: normal;
                   font-weight: normal;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: left;
                   text-transform: normal;
                   letter-spacing: 0px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }
               """

In [13]:
with gr.Blocks(theme=gr.themes.Base(), css=css_template) as iface:
    gr.Markdown("# Exam Generator")
    gr.Markdown(
        "Introducing our innovative exam generation app! This user-friendly tool allows you to create "
        "multiple-choice exams effortlessly. Choose from a variety of pre-designed exams by selecting one "
        "from the dropdown menu and clicking 'Retrieve Exam.' Alternatively, you can harness the power of ChatGPT "
        "to generate custom questions on any subject at your desired grade level. Simply enter the subject "
        "or select it from the dropdown, choose the grade level, and click 'Generate Questions by ChatGPT'. "
        "Once you've answered the questions, click 'Grade Exam' to see your score. "
        "The app will also highlight any questions you answered incorrectly, helping you identify "
        "areas for improvement. Get started today and enhance your learning experience!")

    with gr.Row():
        with gr.Column():
            gr.Markdown("## Pre-Designed Exam")
            dropdown3 = gr.Dropdown(choices=['American Mathematics Competitions 8 (AMC 8)',
                                             'American Mathematics Competitions 10 (AMC 10)',
                                             'AP US History',
                                             'Bible',
                                             'Vocabulary (6th Grade)',
                                             'Vocabulary (8th Grade)',
                                             'Vocabulary (10th Grade)',
                                             'Vocabulary (12th Grade)',
                                             ],
                                    value='American Mathematics Competitions 8 (AMC 8)',
                                    multiselect=False,
                                    label="Exam Title")
            update_btn2 = gr.Button("Retrieve Exam")

        with gr.Column():
            gr.Markdown("## Questions Generated by ChatGPT")
            dropdown1 = gr.Dropdown(choices=['Math',
                                             'Geography',
                                             'U.S. History',
                                             'Science',
                                             'History'],
                                    value='History',
                                    multiselect=False,
                                    allow_custom_value=True,
                                    label="Subject")
            dropdown2 = gr.Dropdown(choices=['First grade',
                                             'Second grade',
                                             'Third grade',
                                             'Fourth grade',
                                             'Fifth grade',
                                             'Sixth grade',
                                             'Junior High School',
                                             'High School',
                                             'Undergraduate',
                                             'Master degree',
                                             'Doctorate',
                                             'Beginner',
                                             'Intermediate',
                                             'Advanced',
                                             'Expert'],
                                    value='Third grade',
                                    multiselect=False,
                                    allow_custom_value=True,
                                    label="Question Grade Level")
            update_btn1 = gr.Button("Generate Questions by ChatGPT")

    gr.Markdown("# Exam")

    inputs = []
    for question in questions:
        inputs.append(gr.Markdown('Questions',
                                  latex_delimiters=[{"left": "$$/", "right": "/$$", "display": False}],
                                  ))
        inputs.append(gr.Radio(choices=question["Choices"],
                               label='',
                               show_label=False,
                               # label=question["Question"]
                               ))

    update_btn1.click(fn=exam_generator1,
                      inputs=[dropdown1, dropdown2],
                      outputs=inputs)
    update_btn2.click(fn=exam_generator2,
                      inputs=[dropdown3],
                      outputs=inputs)

    submit_btn = gr.Button("Grade Exam")

    output2 = gr.Markdown(label="Exam Result",
                          latex_delimiters=[{"left": "$$/", "right": "/$$", "display": False}],
                          )
    submit_btn.click(fn=grade_exam, inputs=inputs[1::2], outputs=output2)

In [14]:
if __name__ == '__main__':
    iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [15]:
iface.close()

Closing server running on port: 7860
